In [ ]:
# OBSOLETE!!! 
# IMPLEMENTING THIS WITH HIPE4ML IN ANOTHER CODE
import ROOT

In [ ]:
f_gen  = ROOT.TFile.Open("~/alice/JPsiDatasets/MC/RawMC/AO2DMC_LHC25b16_0_550369_AOD_001.root")
f_reco = ROOT.TFile.Open("~/alice/O2Physics/PWGJE/Tasks/JPsiWorkDir/JPsiMC/DQEfficiency/AnalysisResults_trees.root")

chain_reco = ROOT.TChain("ReconstructedTrees")

for key in f_reco.GetListOfKeys():
    obj = key.ReadObj()
    if isinstance(obj, ROOT.TDirectoryFile) and obj.GetName().startswith("DF"):
        # Add tree to chain using "file.root/dir/tree" path
        chain.Add(f"{f.GetName()}/{obj.GetName()}/RTDielectron")

print("Total entries:", chain.GetEntries())

# Now you can treat `chain` exactly like a TTree
n_pass = chain.Draw("mass", "mass>3.0 && mass<3.2", "goff")
efficiency = n_pass / chain.GetEntries() if chain.GetEntries() else 0
print(f"Global efficiency: {efficiency:.4f}")

# Get the TTrees (replace with the actual names in your files)
# t_gen  = f_gen.Get("JpsiGenTree")
# t_reco = f_reco.Get("JpsiRecoTree")

# Define fiducial cuts
pt_cut = "pt > 0 and pt < 10"
y_cut  = "abs(y) < 0.9"
fiducial = f"{pt_cut} && {y_cut}"

# Acceptance
total_gen = t_gen.GetEntries("")             # all generated
gen_in_fid = t_gen.GetEntries(fiducial)      # generated inside acceptance
acceptance = gen_in_fid / total_gen

# Efficiency
reco_in_fid = t_reco.GetEntries(fiducial)    # reconstructed inside acceptance
efficiency  = reco_in_fid / gen_in_fid

# Acc × Eff
acc_eff = reco_in_fid / total_gen

print(f"Inclusive Acceptance = {acceptance:.3f}")
print(f"Inclusive Efficiency = {efficiency:.3f}")
print(f"Inclusive Acc × Eff  = {acc_eff:.3f}")


Hello, world!


In [ ]:
# Define histograms
h_gen  = ROOT.TH1F("h_gen",  "Generated; pT; Counts", 10, 0, 10)
h_reco = ROOT.TH1F("h_reco", "Reconstructed; pT; Counts", 10, 0, 10)

# Fill from TTrees
t_gen.Draw("pt >> h_gen", fiducial)
t_reco.Draw("pt >> h_reco", fiducial)

# Efficiency histogram
h_eff = h_reco.Clone("h_eff")
h_eff.Divide(h_gen)

h_eff.Draw()


In [ ]:
# Create histograms (same binning for gen and reco)
pt_bins = [0, 1, 2, 3, 4, 5, 6, 8, 10]  # non-uniform example
h_gen  = ROOT.TH1F("h_gen",  "Generated;p_{T} (GeV/c);Counts", len(pt_bins)-1, array('d', pt_bins))
h_reco = ROOT.TH1F("h_reco", "Reconstructed;p_{T} (GeV/c);Counts", len(pt_bins)-1, array('d', pt_bins))

# Fill generated histogram
t_gen.Draw("pt >> h_gen", fiducial)

# Fill reconstructed histogram with truth match condition
# We require the fiducial cuts to match gen-level kinematics of the candidate
truth_cut = f"{fiducial} && isMCTruthMatched == 1"
t_reco.Draw("pt >> h_reco", truth_cut)

# Create TEfficiency
if ROOT.TEfficiency.CheckConsistency(h_reco, h_gen):
    eff_pt = ROOT.TEfficiency(h_reco, h_gen)
    eff_pt.SetTitle("J/#psi Acceptance #times Efficiency;p_{T} (GeV/c);Acc #times Eff")
    eff_pt.SetStatisticOption(ROOT.TEfficiency.kBUniform)  # binomial errors
else:
    raise RuntimeError("Histograms are not consistent for TEfficiency")

# Draw
c = ROOT.TCanvas("c", "", 800, 600)
eff_pt.Draw("AP")  # "AP" = axis + points
c.SaveAs("acc_eff_pt.pdf")